In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [4]:
df.describe(include=['bool','object'])

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
count,1470,1470,1470,1470,1470,1470,1470,1470,1470
unique,2,3,3,6,2,9,3,1,2
top,No,Travel_Rarely,Research & Development,Life Sciences,Male,Sales Executive,Married,Y,No
freq,1233,1043,961,606,882,326,673,1470,1054


In [5]:
df['Attrition']=df.Attrition.astype('category').cat.codes

In [6]:
df.drop(['EmployeeCount','EmployeeNumber','Over18', 'StandardHours'], axis=1,inplace=True)

In [7]:
categorical_columns=[]

for col in df.columns:
    if df[col].dtypes == object and len(df[col].unique())<=50:
        categorical_columns.append(col)

categorical_columns

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [9]:
from sklearn.preprocessing import LabelEncoder

label=LabelEncoder()

for col in categorical_columns:
    df[col]=label.fit_transform(df[col])

In [10]:
from sklearn.model_selection import train_test_split

X=df.drop('Attrition', axis=1)
y=df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_cf=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),n_estimators=50)
ada_cf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1))

In [14]:
print_score(ada_cf, X_train, y_train, X_test, y_test, train=True)
print_score(ada_cf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 90.57%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.911765    0.855856  0.905734     0.883810      0.902202
recall       0.981243    0.539773  0.905734     0.760508      0.905734
f1-score     0.945229    0.662021  0.905734     0.803625      0.896789
support    853.000000  176.000000  0.905734  1029.000000   1029.000000
_______________________________________________
Confusion Matrix: 
 [[837  16]
 [ 81  95]]

Test Result:
Accuracy Score: 86.39%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.900000   0.512195  0.863946    0.706098      0.846358
recall       0.947368   0.344262  0.863946    0.645815      0.863946
f1-score     0.923077   0.411765  0.863946    0.667421      0.852351
support    380.000000  61.000000  0.863946  441.000000    441.

In [16]:
n_estimators=[int(x) for x in np.linspace(start=50, stop=500, num=10)]
learning_rate=[0.1,0.05, 0.01]

In [19]:
from sklearn.model_selection import RandomizedSearchCV
random_grid={
    'n_estimators': n_estimators,
    'learning_rate' : learning_rate
}

ada_cf= AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),random_state=42)

gb_cv = RandomizedSearchCV(
    estimator=ada_cf,
    scoring = 'f1',
    param_distributions=random_grid,
    n_iter=200,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

gb_cv.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 30 is smaller than n_iter=200. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                                                random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.01],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300, 350, 400, 450,
                                                         500]},
                   random_state=42, scoring='f1', verbose=1)

In [21]:
best_params=gb_cv.best_params_
ada_cf= AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),random_state=42,n_estimators=500,learning_rate=0.1)
ada_cf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.1, n_estimators=500, random_state=42)

In [22]:
print_score(ada_cf, X_train, y_train, X_test, y_test, train=True)
print_score(ada_cf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 89.50%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.895016    0.895349  0.895044     0.895182      0.895073
recall       0.989449    0.437500  0.895044     0.713475      0.895044
f1-score     0.939866    0.587786  0.895044     0.763826      0.879647
support    853.000000  176.000000  0.895044  1029.000000   1029.000000
_______________________________________________
Confusion Matrix: 
 [[844   9]
 [ 99  77]]

Test Result:
Accuracy Score: 87.76%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.891827   0.640000  0.877551    0.765913      0.856994
recall       0.976316   0.262295  0.877551    0.619305      0.877551
f1-score     0.932161   0.372093  0.877551    0.652127      0.854691
support    380.000000  61.000000  0.877551  441.000000    441.